In [ ]:
Testing our model manually.

In [4]:
import joblib
import pandas as pd

# Load saved model
model = joblib.load("artifacts/fraud_model.pkl")

# Create a fake FRAUD transaction
# (sending exact balance → account drained → classic fraud!)
transaction = {
    "step"           : 1,
    "type"           : 4,        # 4 = TRANSFER
    "amount"         : 500000,
    "oldbalanceOrg"  : 500000,   # sender had exactly this amount
    "newbalanceOrig" : 0,        # now zero → drained!
    "oldbalanceDest" : 0,
    "newbalanceDest" : 500000
}

# Calculate engineered features
transaction["balanceErrorOrig"] = transaction["oldbalanceOrg"] - transaction["amount"] - transaction["newbalanceOrig"]
transaction["balanceErrorDest"] = transaction["oldbalanceDest"] + transaction["amount"] - transaction["newbalanceDest"]

# Convert to dataframe
df_input = pd.DataFrame([transaction])

# Predict
prediction  = model.predict(df_input)[0]
probability = model.predict_proba(df_input)[0][1]

print("Transaction Details:")
print(f"   Type    : TRANSFER")
print(f"   Amount  : $500,000")
print(f"   Result  : {'🚨 FRAUD' if prediction == 1 else '✅ LEGITIMATE'}")
print(f"   Confidence : {probability*100:.2f}%")

Transaction Details:
   Type    : TRANSFER
   Amount  : $500,000
   Result  : 🚨 FRAUD
   Confidence : 74.00%


In [3]:
import joblib
import pandas as pd

model = joblib.load("artifacts/fraud_model.pkl")

# Legitimate transaction
# (normal payment — sender still has money left)
transaction = {
    "step"           : 1,
    "type"           : 3,        # 3 = PAYMENT
    "amount"         : 1000,
    "oldbalanceOrg"  : 50000,    # sender had plenty
    "newbalanceOrig" : 49000,    # still has money left
    "oldbalanceDest" : 10000,
    "newbalanceDest" : 11000
}

transaction["balanceErrorOrig"] = transaction["oldbalanceOrg"] - transaction["amount"] - transaction["newbalanceOrig"]
transaction["balanceErrorDest"] = transaction["oldbalanceDest"] + transaction["amount"] - transaction["newbalanceDest"]

df_input = pd.DataFrame([transaction])

prediction  = model.predict(df_input)[0]
probability = model.predict_proba(df_input)[0][1]

print("Transaction Details:")
print(f"   Type       : PAYMENT")
print(f"   Amount     : $1,000")
print(f"   Result     : {'🚨 FRAUD' if prediction == 1 else '✅ LEGITIMATE'}")
print(f"   Confidence : {probability*100:.2f}%")

Transaction Details:
   Type       : PAYMENT
   Amount     : $1,000
   Result     : ✅ LEGITIMATE
   Confidence : 0.00%
